In [6]:
import spacy
import pandas as pd
from numpy import random
import matplotlib.pyplot as plt
import re
import json
from tqdm import tqdm
import jsonlines

# adding special symbol to tokenizer
EOS_re = re.compile(r'</s>')

from spacy.tokenizer import Tokenizer
def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, token_match=EOS_re.match)

nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = custom_tokenizer(nlp)

From MNLI code, read data line by line. 

In [9]:
def load_nli_data(path):
    """
    Load MultiNLI or SNLI data.
    """
    LABEL_MAP = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2}
    
    data = []
    with open(path) as f:
        for line in f:
            loaded_example = json.loads(line)
            if loaded_example["gold_label"] not in LABEL_MAP:
                continue
            loaded_example["label"] = LABEL_MAP[loaded_example["gold_label"]]
            data.append(loaded_example)
            
        random.seed(1)
        random.shuffle(data)
        
    return data

Read MNLI data as JSONL. Split data in 50-50 train-test (finetune-generate). 

In [10]:
# data = load_nli_data('multinli_1/multinli_1_train.jsonl')

# train = pd.DataFrame(data[0:round(len(data)/2)])
# test = pd.DataFrame(data[round(len(data)/2)+1:])

# # output half of data to pre-train, half to generate over
# train.to_json('mnli_bart_train.jsonl', orient ='records', lines=True)
# test.to_json('mnli_bart_test.jsonl', orient ='records', lines=True)

Replicate the "mask-infill" training objective found in BART Denoising task. 
Identical masking to mask_fill objective. Mask hypothesis, then concate premise to masked hypothesis, separated by special token `</s>` used for sentence entailment fine-tuning. 

In [11]:
def mask_hypo(hypo):
    
    sent = nlp(hypo)
    # sample mask span length from poisson with lambda = 3, as in BART denoising paper
    draw = random.poisson(3)
    
    # if masked span is longer than sentence
    if len(sent) - draw <= 0:
        return '<mask>'
    
    start_ind = random.randint(len(sent) - draw)
    
    # insert <mask> 
    if draw == 0:
        return sent[:start_ind].text + ' <mask> ' + sent[start_ind:].text
    
    masked_sent = sent[:start_ind].text + ' <mask> ' + sent[start_ind + draw:].text
    
    return masked_sent

In [379]:
masked_pairs = []
original_pairs = []
for index, row in tqdm(train.iterrows(), total=train.shape[0]):
    masked_hypo = mask_hypo(row['sentence2'])
    
    # appending masked and original premise + </s> + hypo pairs
    masked_pairs.append(row['sentence1'] + ' </s> ' + mask_hypo(row['sentence2']))
    original_pairs.append(row['sentence1'] + ' </s> ' + row['sentence2'])

100%|██████████| 196351/196351 [30:02<00:00, 108.91it/s]


In [380]:
pickle.dump(masked_pairs, open("denoised_input.pkl", "wb"))
pickle.dump(original_pairs, open("denoised_output.pkl", "wb"))

Format the input-output of train-val-test splits for **fairseq** fine-tuning. 

In [3]:
import pickle
import numpy as np

ip = pickle.load(open("denoised_input.pkl", "rb"))
op = pickle.load(open("denoised_output.pkl", "rb"))

ip = np.asarray(ip)
op = np.asarray(op)
 
indices = np.arange(ip.shape[0])
np.random.shuffle(indices)

ip = ip[indices]
op = op[indices]
DIR = "./" #"../Bart/Denoised/"
pickle.dump(indices, open(DIR+"denoised_shuffle_indices.pkl", "wb"))

for i in range(len(ip)):
    ip[i] = ' '.join(ip[i].split())
    op[i] = ' '.join(op[i].split())

with open(DIR+"train.source", "w") as f1, open(DIR+"train.target", "w") as f2:
    for i in range(len(ip[:150000])):
        f1.write(ip[i].strip()+'\n')
        f2.write(op[i].strip()+'\n')


with open(DIR+"val.source", "w") as f1, open(DIR+"val.target", "w") as f2:
    for i in range(len(ip[150000:22000])):
        f1.write(ip[21000+i].strip()+'\n')
        f2.write(op[21000+i].strip()+'\n')

with open(DIR+"test.source", "w") as f1, open(DIR+"test.target", "w") as f2:
    for i in range(len(ip[22000:])):
        f1.write(ip[22000+i].strip()+'\n')
        f2.write(op[22000+i].strip()+'\n')

In [ ]:
lengths = [] 
for i in tqdm(range(len(train))):
    lengths.append(len(nlp(train['sentence2'][i])))
    
plt.hist(lengths)

  1%|          | 2086/196351 [00:08<13:39, 236.98it/s]